# Geolokalizacja użytkowników tweetera #sczepimysie

In [10]:
import pandas as pd

In [11]:
df_users = pd.read_json('data/users_data_2021_12_09.json')

Wyrzucamy śmieci

In [12]:
# Drop columns, where there are no values:
df_users.dropna(how='all', axis=1, inplace=True)
# Drop columns, where there's only one value
df_users.drop([col for col in df_users.columns if len(pd.unique(df_users[col].astype(str))) == 1], axis=1, inplace=True)
df_users.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 9758 entries, 0 to 9757
Data columns (total 35 columns):
 #   Column                             Non-Null Count  Dtype              
---  ------                             --------------  -----              
 0   id                                 9758 non-null   int64              
 1   id_str                             9758 non-null   int64              
 2   name                               9758 non-null   object             
 3   screen_name                        9758 non-null   object             
 4   location                           9758 non-null   object             
 5   description                        9758 non-null   object             
 6   url                                3539 non-null   object             
 7   followers_count                    9758 non-null   int64              
 8   normal_followers_count             9758 non-null   int64              
 9   friends_count                      9758 non-null   i

Patrzymy jakie kolumny mogą się przydać

In [13]:
df_users.columns

Index(['id', 'id_str', 'name', 'screen_name', 'location', 'description', 'url',
       'followers_count', 'normal_followers_count', 'friends_count',
       'listed_count', 'created_at', 'favourites_count', 'geo_enabled',
       'verified', 'statuses_count', 'media_count', 'is_translation_enabled',
       'profile_background_color', 'profile_background_tile',
       'profile_link_color', 'profile_sidebar_border_color',
       'profile_sidebar_fill_color', 'profile_text_color',
       'profile_use_background_image', 'has_extended_profile',
       'default_profile', 'pinned_tweet_ids', 'pinned_tweet_ids_str',
       'has_custom_timelines', 'advertiser_account_type',
       'advertiser_account_service_levels', 'profile_interstitial_type',
       'translator_type', 'withheld_in_countries'],
      dtype='object')

W zasadzie tylko location. geo_enabled to bool i niezależnie od jej wartości, 'location' może być puste lub nie. Reszta to zbędne dane na temat profilu.

#### Kolumna 'location'

In [14]:
df_users.location

0                 Polska
1                       
2                Arcelin
3           Łódź, Poland
4                       
              ...       
9753           Stockholm
9754    Around the world
9755              CANADA
9756      Washington, DC
9757     London, England
Name: location, Length: 9758, dtype: object

Z tej kolumny tyle wierszy nie jest pustych:

In [15]:
(df_users.location.str.len()>0).sum()

6664

Z czego tyle unikatowych

In [16]:
unique_locations = df_users[df_users.location.str.len()>0].location.unique()
len(unique_locations)

2538

Na Nominatimie bardzo łatwo dostać bana za nadużywanie API:

https://operations.osmfoundation.org/policies/nominatim/

Więc spróbujmy na starcie odfiltrować chociaż trochę dane, które w oczywisty sposób nas nie interesują. Najpierw przejrzyjmy dane:

In [17]:
unique_locations = unique_locations.tolist()

In [18]:
unique_locations

['Polska',
 'Arcelin',
 'Łódź, Poland',
 'Rawa Mazowiecka Przemysłowa 2a',
 'Plecak pełen cashu',
 'Poznań, Polska',
 'Bolesławiec, Polska',
 'Katowice, Polska',
 'Zalasewo',
 'Wołoszczyzna',
 'Zielona Góra, Polska',
 'Białystok/Olsztyn, Polska',
 'Lublin',
 'Warsaw, Poland',
 'Greece',
 'Tuchów, Polska',
 'Gdynia, Polska',
 'Tomaszów Mazowiecki, Polska',
 'Wodzierady, Polska',
 'Kraków, Polska',
 'Warszawa, Polska',
 'Piwo ',
 'Toruń, Gdynia, Kraków, Poznań ',
 'Barczewo',
 'Gliwice, Polska',
 'Gdańsk, Polska',
 'Schlesien, Polen',
 'Wołomin, Polska',
 'Mazowieckie, Polska',
 'Wejherowo, Polska',
 'Warszawa ',
 'Częstochowa, Śląskie',
 'Malmo, Sweden',
 'Wrocław',
 'Tel Aviv',
 '38-722 Olszanica 222',
 'Warszawa',
 'Bydgoszcz, Polska',
 'Mont Blanc',
 'Zachodniopomorskie, Polska',
 'Poland',
 'Wrocław, Polska',
 'Sieradz, Polska',
 '11-430 Korsze, Dubliny 16',
 'Bielsko-Biała',
 'Katowice',
 'Wrocław, Poland',
 'Przytuły Stare',
 'Przeworsk',
 'Zwoleń, Polska',
 'Ard Skellig',
 'Krako

Większość wygląda jakby mogła być z Polski, więc odsiewamy te z całej reszty świata, żeby minimalizować użycie API.

In [19]:
!pip install country-list

In [21]:
from country_list import countries_for_language
import yaml

countries = list(map(lambda c: c[1], countries_for_language('en')))
countries.remove('Poland')

foreign_cities = []
with open("data/biggest_cities.yaml", "r") as stream:
    try:
        foreign_cities = yaml.safe_load(stream)['biggest_cities']
    except yaml.YAMLError as exc:
        print(exc)

blacklist = countries + foreign_cities
blacklist[:5]

['Afghanistan', 'Åland Islands', 'Albania', 'Algeria', 'American Samoa']

In [22]:
def remove_from_list(remove_from_this, remove_those):
    remove_those = set(remove_those)
    remove_bin = set()
    for location in remove_from_this:
        for blacklist_el in remove_those:
            if blacklist_el in location:
                remove_bin.add(location)
    for el in list(remove_bin):
        remove_from_this.remove(el)
    return remove_from_this


In [23]:
unique_locations = remove_from_list(unique_locations, blacklist)

In [24]:
len(unique_locations)

2276

In [25]:
unique_locations

['Polska',
 'Arcelin',
 'Łódź, Poland',
 'Rawa Mazowiecka Przemysłowa 2a',
 'Plecak pełen cashu',
 'Poznań, Polska',
 'Bolesławiec, Polska',
 'Katowice, Polska',
 'Zalasewo',
 'Wołoszczyzna',
 'Zielona Góra, Polska',
 'Białystok/Olsztyn, Polska',
 'Lublin',
 'Warsaw, Poland',
 'Tuchów, Polska',
 'Gdynia, Polska',
 'Tomaszów Mazowiecki, Polska',
 'Wodzierady, Polska',
 'Kraków, Polska',
 'Warszawa, Polska',
 'Piwo ',
 'Toruń, Gdynia, Kraków, Poznań ',
 'Barczewo',
 'Gliwice, Polska',
 'Gdańsk, Polska',
 'Schlesien, Polen',
 'Wołomin, Polska',
 'Mazowieckie, Polska',
 'Wejherowo, Polska',
 'Warszawa ',
 'Częstochowa, Śląskie',
 'Wrocław',
 'Tel Aviv',
 '38-722 Olszanica 222',
 'Warszawa',
 'Bydgoszcz, Polska',
 'Mont Blanc',
 'Zachodniopomorskie, Polska',
 'Poland',
 'Wrocław, Polska',
 'Sieradz, Polska',
 '11-430 Korsze, Dubliny 16',
 'Bielsko-Biała',
 'Katowice',
 'Wrocław, Poland',
 'Przytuły Stare',
 'Przeworsk',
 'Zwoleń, Polska',
 'Ard Skellig',
 'Krakow, Poland',
 'Podkarpackie, P

In [26]:
len(df_users[df_users['location'].str.lower().isin([s.lower() for s in unique_locations])])

6210

Odpytywanie API o lokacje

In [27]:
from geopy.geocoders import Nominatim

#Na początek próba połączenia
geolocator = Nominatim(timeout=10, user_agent = "vaccination_data")

location = geolocator.geocode('Around the world')
print(location)
print((location.latitude, location.longitude))

Around the World, Whistler Olympic Park, Area D (Elaho/Garibaldi), Squamish-Lillooet Regional District, British Columbia, Canada
(50.1308051, -123.1213132)


In [28]:
# Teraz próba na danych z kosmosu co zwraca Nominatim
with open("data/example_trash.yaml", "r", encoding='utf-8') as stream:
    try:
        invalid_locations = yaml.safe_load(stream)['invalid_locations']
    except yaml.YAMLError as exc:
        print(exc)
invalid_locations[:5]

['Universe', 'Europe', 'Texas', 'spółka zło dundersztyca', 'Ard skellig']

Próbka czy API zwraca sensowne wartości dla bezsensownego wejścia

In [30]:
from time import sleep
import pickle
import os

dict_request_response = {} if not os.path.exists('data/locations_geocoded.pkl') \
    else pickle.load(open('data/locations_geocoded.pkl', 'rb+'))
for inv_loc in invalid_locations:
    if inv_loc not in dict_request_response:
        dict_request_response[inv_loc] = geolocator.geocode(inv_loc)
        sleep(2)
    print(f'Dla zapytania "{inv_loc}", odpowiedź API to: {dict_request_response[inv_loc]}')
        

Dla zapytania "Universe", odpowiedź API to: Universe, Grønvej, Elsmark, Svenstrup, Sønderborg Kommune, Region Syddanmark, 6430, Danmark
Dla zapytania "Europe", odpowiedź API to: أوروبا
Dla zapytania "Texas", odpowiedź API to: Texas, United States
Dla zapytania "spółka zło dundersztyca", odpowiedź API to: None
Dla zapytania "Ard skellig", odpowiedź API to: None
Dla zapytania "Bradenton", odpowiedź API to: Bradenton, Manatee County, Florida, United States
Dla zapytania "Lemingrad", odpowiedź API to: None
Dla zapytania "Tam gdzie jest ciekawie", odpowiedź API to: None
Dla zapytania "tuitam", odpowiedź API to: Tuitam - pierogarnia chińska, Wielkopolska, Wielki Kack, Gdynia, województwo pomorskie, 81-587, Polska
Dla zapytania "Bayern", odpowiedź API to: Bayern, Deutschland
Dla zapytania "Londyn", odpowiedź API to: London, Greater London, England, United Kingdom
Dla zapytania "Worldwide", odpowiedź API to: Worldwide, Edward Street, Ladywood, Park Central, Birmingham, West Midlands Combined A

Radzi sobie tak, że będzie to można odsiać. Zapytajmy zatem o całość, póki co respektując zasady API, by nie puszczać requestów częściej niż co sekundę:

In [31]:
response_cache = dict_request_response
for i, inv_loc in enumerate(unique_locations):
    if inv_loc not in response_cache:
        response_cache[inv_loc] = geolocator.geocode(inv_loc)
        print(f'token{i+1}: Dla zapytania "{inv_loc}", odpowiedź API to: {response_cache[inv_loc]}')
        sleep(2)
    else:
        print(f'cachedToken{i+1}: Dla zapytania "{inv_loc}", odpowiedź API to: {response_cache[inv_loc]}')

cachedToken1: Dla zapytania "Polska", odpowiedź API to: Polska
cachedToken2: Dla zapytania "Arcelin", odpowiedź API to: Arcelin, gmina Płońsk, powiat płoński, województwo mazowieckie, Polska
cachedToken3: Dla zapytania "Łódź, Poland", odpowiedź API to: Łódź, województwo łódzkie, Polska
cachedToken4: Dla zapytania "Rawa Mazowiecka Przemysłowa 2a", odpowiedź API to: Jednostka Ratowniczo-Gaśnicza PSP w Rawie Mazowieckiej, 2A, Przemysłowa, Osiedle 9 Maja, Osada Dolna, Rawa Mazowiecka, powiat rawski, województwo łódzkie, 96-200, Polska
cachedToken5: Dla zapytania "Plecak pełen cashu", odpowiedź API to: None
cachedToken6: Dla zapytania "Poznań, Polska", odpowiedź API to: Poznań, powiat poznański, województwo wielkopolskie, Polska
cachedToken7: Dla zapytania "Bolesławiec, Polska", odpowiedź API to: Bolesławiec, powiat bolesławiecki, województwo dolnośląskie, Polska
cachedToken8: Dla zapytania "Katowice, Polska", odpowiedź API to: Katowice, Górnośląsko-Zagłębiowska Metropolia, województwo śląs

Zapis danych

In [277]:
import pickle
 
pickle.dump(response_cache, open('data/locations_geocoded.pkl', 'wb+'))

#### Ekstrakcja id użytkowników, o których mamy informacje nt. lokalizacji

In [50]:
known_users_ids = set(df_users.id.tolist())
df_users_locations = df_users[['id', 'location']]

#### Wczytywanie i ekstrakcja danych na temat tweetów, ich autorów i estymowanego nastwawienia

In [60]:
df_tweets = pd.read_json('data/tweets_data_full_labeled_2021_12_09.json')
df_tweets.columns, len(df_tweets)

(Index(['created_at', 'id', 'id_str', 'full_text', 'truncated', 'main_hashtags',
        'user_mention', 'in_reply_to_status_id', 'in_reply_to_status_id_str',
        'in_reply_to_user_id', 'in_reply_to_user_id_str',
        'in_reply_to_screen_name', 'user_id', 'user_id_str', 'geo',
        'coordinates', 'place', 'contributors', 'is_quote_status',
        'retweet_count', 'favorite_count', 'reply_count', 'quote_count',
        'conversation_id', 'favorited', 'retweeted', 'possibly_sensitive',
        'lang', 'bert_proba_0', 'bert_proba_1', 'bert_label',
        'xg_boost_proba_0', 'xg_boost_proba_1', 'xg_boost_labels', 'cluster',
        'sentiment_tag', 'stance_tag',
        'SentimentPL_clear_full_text_without_hashtags',
        'SentimentPL_clear_full_text_with_hashtags',
        'SentimentPL_raw_full_text', 'y2classes_numeric', 'y2classes_str',
        'y3classes_numeric', 'y3classes_str'],
       dtype='object'),
 36787)

In [59]:
tweets_users = set(df_tweets.user_id.tolist())

In [56]:
len(tweets_users), len(known_users_ids), len(tweets_users - known_users_ids)

(7026, 9758, 850)

In [77]:
tweets_known_users_location = df_tweets[df_tweets.user_id.isin(known_users_ids)]
relevant_tweets_attr = tweets_known_users_location[['user_id', 'y2classes_numeric', 'y3classes_numeric']]

#### JOIN między tweetami i lokacjami

In [94]:
tweet_user_location =relevant_tweets_attr.merge(df_users_locations, left_on='user_id', right_on='id')\
    .drop(['id'], axis=1)\
    .drop_duplicates()
tweet_user_location.rename({'location': 'location_by_user'}, axis=1, inplace=True)
tweet_user_location.head()


,user_id,y2classes_numeric,y3classes_numeric,location_by_user
0,837675106285551618,-1,0,"Wielkopolskie, Polska"
1,837675106285551618,-1,-1,"Wielkopolskie, Polska"
11,837675106285551618,1,1,"Wielkopolskie, Polska"
13,837675106285551618,1,-1,"Wielkopolskie, Polska"
42,1020384619,-1,-1,"Bydgoszcz, Kujawsko-Pomorskie"


#### JOIN między tabelą tweet_lokacja, a informacjami z API Normatim

In [114]:
tweet_user_location['location_by_API'] = list(
        map(
            lambda loc: response_cache.get(loc, None), 
            list(tweet_user_location.location_by_user)
        )
    )
tweet_user_location.reset_index(drop=True, inplace=True)

#### Zapis danych

------------------------
Nie można tego zapisać pod postacią DataFrame'a, bo obiekty Location nie chcą się dobrze serializować. Wykorzystam pickle.

In [117]:
tweet_user_location_dict = tweet_user_location.to_dict(orient='records')[0]['location_by_API']